In [ ]:
import os
import subprocess
import re
import sys
from showenv import source_directory, destination_directory

In [ ]:
def extract_subtitles(mkv_file, output_dir):
    """
    Extracts PGS subtitles from an MKV file.
    
    Parameters:
    mkv_file (str): The path to the MKV file.
    output_dir (str): The directory to save the extracted subtitles.
    
    Returns:
    str: The path to the extracted subtitle file.
    """
    # Get the correct subtitle track index
    subtitle_stream_index = get_subtitle_stream_index(mkv_file)
    
    if subtitle_stream_index is None:
        print(f"No PGS subtitle stream found in {mkv_file}.")
        return None

    # Create the output directory if it does not exist
    os.makedirs(output_dir, exist_ok=True)

    output_file = os.path.join(output_dir, 'subtitles.sup')
    
    # Command to extract the subtitles using mkvextract
    command = ['mkvextract', 'tracks', mkv_file, f'{subtitle_stream_index}:{output_file}']
    
    try:
        subprocess.run(command, check=True)
        print(f"Subtitles extracted successfully to {output_file}")
        return output_file
    except subprocess.CalledProcessError as e:
        print(f"Error extracting subtitles from {mkv_file}: {e}")
        return None

In [ ]:
def get_subtitle_stream_index(mkv_file):
    """
    Get the index of the PGS subtitle track from the MKV file.
    
    Parameters:
    mkv_file (str): The path to the MKV file.
    
    Returns:
    str: The subtitle track index.
    """
    command = ['mkvmerge', '-i', mkv_file]
    result = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    output = result.stdout
    return 4
    print(output)

    sys.exit()

    for line in output.splitlines():
        if 'PGS' in line:
            parts = line.split(':')
            print(parts)
            # track_index = parts[0].strip().split(' ')[1]
            # return track_index

    return None

In [ ]:
def process_videos(source_dir, destination_dir):
    """
    Process all video files in the source directory to extract PGS subtitles.
    """
    # Define the range of seasons (e.g., Season 1 to Season 7)
    seasons = sorted([f"Season {i}" for i in range(1, 8)])

    # Loop through each season folder
    for season in seasons:
        season_path = os.path.join(source_dir, season)

        # Check if the season directory exists
        if not os.path.isdir(season_path):
            print(f"Directory {season_path} does not exist, skipping.")
            continue

        # Create corresponding destination directory for the season
        destination_season_path = os.path.join(destination_dir, season)
        os.makedirs(destination_season_path, exist_ok=True)

        # Get sorted list of files in the season folder
        files = sorted(os.listdir(season_path))

        # Process each file in the season folder
        for filename in files:
            # Look for complete .mkv files
            if filename.endswith('.mkv') and not filename.endswith('.mkv.part'):

                pattern = r"S\d{2}E\d{2} - [^()]+"

                match = re.search(pattern, filename)
                if match:
                    modified_filename = match.group(0)
                    # print(episode_title)  # Output: S01E01 - Fundraiser
                # else:
                    # print("Pattern not found in the filename.")

                source_file = os.path.join(season_path, filename)

                # Create a directory for the episode
                output_directory = os.path.join(destination_season_path, modified_filename.strip())

                # Ensure the output directory exists
                os.makedirs(output_directory, exist_ok=True)

                # Extract PGS subtitles
                extract_subtitles(source_file, output_directory)


In [ ]:
# Call the function to burn subtitles
process_videos(source_directory, destination_directory)
